In [4]:
import pandas as pd

In [5]:
data=pd.read_csv("patstat_pat_no.csv")

In [6]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [7]:
import mysql.connector

In [8]:
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}


In [10]:
data["docdb_family_id"]

0           574528.0
1           574586.0
2           574660.0
3          1226456.0
4          1226485.0
             ...    
229277    49776606.0
229278    49881149.0
229279    49882718.0
229280    49998818.0
229281    49999637.0
Name: docdb_family_id, Length: 229282, dtype: float64

In [11]:
tuple(data["docdb_family_id"])

(574528.0,
 574586.0,
 574660.0,
 1226456.0,
 1226485.0,
 1236211.0,
 1236234.0,
 1236542.0,
 1239366.0,
 1239622.0,
 1239657.0,
 1239741.0,
 1239749.0,
 1239760.0,
 1340219.0,
 1340485.0,
 1340488.0,
 1340519.0,
 1340681.0,
 1341021.0,
 1762664.0,
 1763206.0,
 3411930.0,
 3460581.0,
 3460621.0,
 3460833.0,
 3460835.0,
 3460912.0,
 3460975.0,
 3461011.0,
 3461021.0,
 3461121.0,
 3461123.0,
 3478350.0,
 3478351.0,
 3478506.0,
 3478541.0,
 3478788.0,
 3478811.0,
 3478851.0,
 3478868.0,
 3478887.0,
 3479022.0,
 3479063.0,
 3479143.0,
 3479187.0,
 3479252.0,
 3479254.0,
 3479275.0,
 3479294.0,
 3479303.0,
 3479359.0,
 3479376.0,
 3479408.0,
 3479415.0,
 3479450.0,
 3479467.0,
 3479475.0,
 3479498.0,
 3479517.0,
 3479590.0,
 3479671.0,
 3479746.0,
 3479774.0,
 3479780.0,
 3479803.0,
 3479805.0,
 3479828.0,
 3479832.0,
 3479837.0,
 3479858.0,
 3479871.0,
 3479909.0,
 3479918.0,
 3479935.0,
 3479986.0,
 3480016.0,
 3480018.0,
 3480053.0,
 3480092.0,
 3480157.0,
 3480195.0,
 3480208.0,
 348023

In [12]:
#Fetching all the similar patents (as earliest patent publication ids) with a given family id from the database
conn = mysql.connector.connect(**db_config)

cursor = conn.cursor(dictionary=True)
docbfid=tuple(data["docdb_family_id"])
query = f"""
SELECT    
	t1.docdb_family_id,
    t1.earliest_pat_publn_id
FROM
    patstat_s24.tls201_appln t1
WHERE
	t1.docdb_family_id IN {docbfid};
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [13]:
data2=pd.DataFrame(results)

In [14]:
data2.head()

,docdb_family_id,earliest_pat_publn_id
0,19188314,286949845
1,19774416,287579451
2,8861213,289129312
3,7681129,288039900
4,19184257,287898353


In [15]:
#example of all the patents with family id=574528
data2[data2["docdb_family_id"]==574528]

,docdb_family_id,earliest_pat_publn_id
55002,574528,381963844
253355,574528,307102426
363431,574528,387115317
472031,574528,307102421
587183,574528,307102423
760005,574528,307102420
797370,574528,403919347
797408,574528,403922357
797541,574528,307102419
849929,574528,307102424


In [16]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [17]:
#after fetching the similar patents for all the family ids in our dataset(data) we get dataset(data2) with length 1340152
print(len(data))
print(len(data2))

229282
1340152


In [18]:
#performing inner join between two datasets
merged_df = pd.merge(data,data2, on=['docdb_family_id', 'earliest_pat_publn_id'], how='inner')

In [19]:
len(merged_df)

229273

In [20]:
#difference gives the number of patent families for which representative patent was not the part of similar patents fetched by database
len(data)-len(merged_df)

9

In [21]:
#to check which patents did not align 
outer_merged_df = pd.merge(data,data2, on=['docdb_family_id', 'earliest_pat_publn_id'], how='outer', indicator=True)
outer_merged_df.head(100)

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,_merge
0,574528.0,307102419.0,NaN,NaN,NaN,right_only
1,574528.0,307102420.0,NaN,NaN,NaN,right_only
2,574528.0,307102421.0,1985-04-10,EP,Y,both
3,574528.0,307102423.0,NaN,NaN,NaN,right_only
4,574528.0,307102424.0,NaN,NaN,NaN,right_only
...,...,...,...,...,...,...
95,1340488.0,301941691.0,NaN,NaN,NaN,right_only
96,1340488.0,301941692.0,1991-02-20,EP,N,both
97,1340488.0,301941694.0,NaN,NaN,NaN,right_only
98,1340519.0,299341298.0,NaN,NaN,NaN,right_only


In [22]:
omitted_df = outer_merged_df[outer_merged_df['_merge'] == 'left_only']
omitted_df.head(100)

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,_merge
28982,4151350.0,302315673.0,1990-10-19,EP,Y,left_only
29299,4154642.0,301078296.0,1993-03-16,EP,Y,left_only
30736,4173026.0,298860978.0,1994-08-18,EP,Y,left_only
663878,25643279.0,387396958.0,1987-05-15,EP,Y,left_only
667223,25677152.0,303057982.0,1991-10-04,EP,Y,left_only
668239,25685277.0,300335060.0,1994-03-07,EP,Y,left_only
737026,27153130.0,301441222.0,1991-10-30,EP,Y,left_only
739665,27204425.0,300681450.0,1992-11-07,EP,Y,left_only
979802,37106903.0,303002833.0,1990-07-26,EP,N,left_only


In [23]:
len(omitted_df)

9

In [24]:
#to fetch the similar patents for the patent family ids for which the match was not found

conn = mysql.connector.connect(**db_config)

cursor = conn.cursor(dictionary=True)
docbfid_omitted=tuple(omitted_df["docdb_family_id"])
query = f"""
SELECT    
	t1.docdb_family_id,
    t1.earliest_pat_publn_id
FROM
    patstat_s24.tls201_appln t1
WHERE
	t1.docdb_family_id IN {docbfid_omitted};
"""
cursor.execute(query)
results2 = cursor.fetchall()
cursor.close()

True

In [25]:
data_omitted=pd.DataFrame(results2)
data_omitted.head()

""


In [25]:
#it shows that for all the patents family ids that were not matched we do not have data about those family ids in our database.